# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

### Import Libraries

Import necessary libraries

In [4]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [1]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [6]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [7]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [9]:
def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    
    data = dataset_options()
    paths = []
    if data['ravdess']:
        paths.append("..\Datasets\RAVDESS\*\Actor_*\*.wav")
    elif data['ravdess_speech']:
        paths.append("..\Datasets\RAVDESS\Speech\Actor_*\*.wav")
    elif data['ravdess_song']:
        paths.append("..\Datasets\RAVDESS\Song\Actor_*\*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion according to filename. dictionary emotions is defined above.
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("..\Datasets\TESS\*AF_*\*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}

In [10]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 356.2415828704834 seconds ---


In [11]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [12]:
X.shape, y.shape

((5252, 40), (5252, 1))

In [13]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [14]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [15]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-592.968567,55.893475,-19.836720,15.536560,-4.794338,-0.364650,-16.015184,-4.505675,-22.446758,-5.302586,...,-6.580008,-2.471473,-3.679453,-0.385655,1.370497,5.212709,4.105060,8.528406,11.643023,neutral
1,-572.270142,54.515331,-20.434851,15.288507,-3.772105,-1.746776,-16.040491,-3.834324,-21.938566,-0.855068,...,-6.469828,-2.460897,-4.112655,0.497339,2.076435,4.925778,4.332017,9.669187,11.968567,neutral
2,-579.631592,54.810806,-15.795018,17.760878,-5.204145,0.406047,-13.555395,-5.260450,-20.265804,-4.165936,...,-5.041465,-2.006429,-3.473266,-0.836756,1.329681,3.319666,5.729831,8.900810,9.879780,neutral
3,-581.533875,53.867306,-19.620136,17.580286,-6.131991,2.285235,-16.539356,-4.878609,-21.178497,-2.921410,...,-5.276451,-2.091552,-4.794956,0.809350,1.387241,4.056455,6.014116,9.287455,11.116165,neutral
4,-593.011841,61.859417,-16.897928,17.581987,-2.287433,0.082139,-16.137861,-0.824916,-19.867085,-3.405410,...,2.587166,3.317116,-4.654195,-4.810930,-3.983778,1.917435,5.603800,10.190963,11.394855,calm


## Shuffling data

In [16]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [17]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")